In [1]:
from PIL import Image 
import PIL 
from pdf2image import convert_from_path
import os
import threading
import PyPDF2
import pandas as pd
from unidecode import unidecode
import re
import ast
from neo4j import GraphDatabase
import csv

In [2]:
# extract_files = os.listdir("data/")
# try:
#     extract_files.remove(".DS_Store")
# except:
#     pass
# extract_files.sort()


# for extract_file in extract_files:
#     with open(f"data/{extract_file}", "rb") as file:
#         reader = PyPDF2.PdfReader(file)
#         data = []
        
#         # Loop through the pages and extract text
#         for page_num in range(len(reader.pages)):
#             page = reader.pages[page_num]
#             text = page.extract_text().strip()
#             data.append([page_num + 1, text])

#     csv_filename = f"extracts/{extract_file.split('.')[0]}.csv"
#     with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerow(["page", "text"])
#         writer.writerows(data)

In [4]:
def search(seekable_name):
    extract_files = os.listdir("extracts/")
    try:
        extract_files.remove(".DS_Store")
    except:
        pass
    extract_files.sort()

    try:
        with open(f"result/{seekable_name}.html", "w") as f:
            f.write("")
    except:
        pass
    
    uni_seekable_name = unidecode(seekable_name).lower()

    if "," in uni_seekable_name:
        nom = uni_seekable_name.split(",")[0].strip()
        prenoms = uni_seekable_name.split(",")[1].strip()
        prenoms = re.sub(r"[ \-\(\)\/\.]", " ", prenoms)
        prenoms = re.sub(" +", " ", prenoms)
        prenoms = prenoms.strip().split(" ")
        prenoms = [prenom for prenom in prenoms if len(prenom) > 2] # suppression si juste initiales
    else:
        nom = uni_seekable_name.strip()
        prenoms = []

    print(f'nom: {nom}')
    print(f'prenom: {prenoms}')


    with open(f"results/{seekable_name}.html", "a") as f:
        for extract_file in extract_files:
            year = extract_file[0:4]
            title_year = f"<h1>{year}</h1>"
            f.write(f"\n{title_year}\n\n")
            df = pd.read_csv(f"extracts/{extract_file}")

            for i in range(len(df)):
                # lignes = ast.literal_eval(df["ocr_text"][i])
                lignes = df["text"][i]
                num_page = df["page"][i]
                page_link = f"<a href='file:///Users/antoinelavacquery/Annuaires/{year}.pdf#page={num_page}'>page {num_page}</a>"

                if not isinstance(lignes, str):
                    lignes = " "
                
                lignes = lignes.split('\n')

                for ligne in lignes:
                    lower = ligne.lower()
                    uni = unidecode(lower)
                    clean = uni.replace(" ", "").replace(".", "").replace(",", "").replace(";", "")

                    if "," in uni_seekable_name and prenoms:
                        found = False
                        for prenom in prenoms:
                            if prenom in clean and nom in clean:
                                found = True
                        if found:
                            f.write(f"{ligne} -> {page_link}<br>\n")
                            print(f"{ligne} -> {page_link}<br>\n")
                    else:
                        if nom in clean:
                            f.write(f"{ligne} -> {page_link}<br>\n")
                            print(f"{ligne} -> {page_link}<br>\n")


In [5]:
def get_neo4j_results_of(query):
    URI = AURA_CONNECTION_URI
    USER = AURA_USERNAME
    PASS = AURA_PASSWORD

    # Driver instantiation
    driver = GraphDatabase.driver(URI, auth=(USER, PASS))

    # Create a driver session
    with driver.session() as session:
        # Use .data() to access the results array
        results = session.run(query)
        results_data = results.data()
        return(results_data)
    driver.close()

In [6]:
query = """
MATCH (h:HOMME)-[e:EXERCE]->(a:ACTIVITÉ {nom:'Agent de change'})
WHERE (e.date_début.year >= 1815 AND e.date_début.year <= 1852)
OR (e.date_fin.year >= 1815 AND e.date_fin.year <= 1852)
RETURN h
"""

results = get_neo4j_results_of(query)
all = [result["h"]["nom"] for result in results]

for name in all:
    search(name)

# search("Bouchet, Benoît")

nom: paganucci
prenom: ['aime']
nom: ferrand
prenom: ['frederic']
Labouroir ,Barthe etFayre . Ferlay (Al.),papetier ,r.del'Aumône ,4.Favre (Fleury ),médecin ,pl.Confort ,5.Ferrand (Frédéric ),agent dechange ,portFayre (M.),tireurd'or,r.Plat -d'Argent ,19. Saint -Clair,19.Favre C.),m.mercier ,r.desFargues ,4.Ferrand (Isaac-Louis),agent dechange ,Favre (E.),propriét .,pl.Croix -Pàquet ,6. port Saint -Clair ,19. -> <a href='file:///Users/antoinelavacquery/Annuaires/1838.pdf#page=436'>page 436</a><br>

Fayolle (J.-A.),commis ,r.Romarin ,11.Ferrand (Frédéric ),agent dechange ,portFayolle (Ve),rentière ,àChoulans ,5. S.-Clair ,19, -> <a href='file:///Users/antoinelavacquery/Annuaires/1839.pdf#page=459'>page 459</a><br>

Boursier ,M.Ferrand (Frédéric ),portSaint -Clair,19. -> <a href='file:///Users/antoinelavacquery/Annuaires/1850.pdf#page=137'>page 137</a><br>

Boursier ,M.Ferrand (Frédéric ),portSaint -Clair,19. -> <a href='file:///Users/antoinelavacquery/Annuaires/1852.pdf#page=210'>page 2